In [ ]:
!pip install bloscpack

     |████████████████████████████████| 102kB 6.0MB/s 
     |████████████████████████████████| 819kB 9.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for blosc: filename=blosc-1.9.2-cp36-cp36m-linux_x86_64.whl size=476058 sha256=61757b8645aa8739e499a7d4bcc7b910783e980b232ab08e7b62999c0c894426
  Stored in directory: /root/.cache/pip/wheels/d4/28/7c/84ad4e27f91d46943386ad9ca3e3494184cf9cffc55d6f9973
Successfully built blosc
  Created wheel for bloscpack: filename=bloscpack-0.16.0-cp36-none-any.whl size=56335 sha256=9c4005613a90dc64b5a9a21cdaff035a190c71404c174e5d85229d76348843c8
  Stored in directory: /root/.cache/pip/wheels/38/07/a7/6eb1c2ee35fb49ff7376d37c00142a73751b34da37d935cfdd
Successfully built bloscpack


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F


import random
from tqdm import tqdm
import numpy as np
import bloscpack as bp
from sklearn import metrics
import time
from datetime import timedelta

from torchvision import models

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Data

In [ ]:
master_X = bp.unpack_ndarray_from_file('/content/drive/MyDrive/统计麻将/new_small_input/input_X_0_100136.nosync.blp')
master_Y = bp.unpack_ndarray_from_file('/content/drive/MyDrive/统计麻将/new_small_input/input_Y_0_100136.nosync.blp')
master_Y = np.where(master_Y==1)[1]
print(master_X.shape, master_Y.shape)

(100136, 50, 4, 34) (100136,)


In [ ]:
# split training and validation set
n = len(master_X)
train_size = int(0.8*n)
train_idx = random.sample(range(n),train_size)
val_idx = [i for i in range(n) if i not in train_idx]

train_X = torch.from_numpy(master_X[train_idx,:,:,:]).type(torch.FloatTensor)
train_Y = torch.from_numpy(master_Y[train_idx]).type(torch.LongTensor)
val_X = torch.from_numpy(master_X[val_idx,:,:,:]).type(torch.FloatTensor)
val_Y = torch.from_numpy(master_Y[val_idx]).type(torch.LongTensor)
print(train_X.shape,train_Y.shape,val_X.shape,val_Y.shape)

# Transform into dataloader
train_dataset = TensorDataset(train_X, train_Y)
val_dataset = TensorDataset(val_X, val_Y)

train_dataloader = DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=2)
val_dataloader = DataLoader(val_dataset,batch_size=128,shuffle=True,num_workers=2)

torch.Size([80108, 50, 4, 34]) torch.Size([80108]) torch.Size([20028, 50, 4, 34]) torch.Size([20028])


# Model

In [ ]:
import torch
import torch.nn as nn
from typing import Union, List, Dict, Any, cast


__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}


class VGG(nn.Module):

    def __init__(
        self,
        features: nn.Module,
        num_classes: int = 34,
        init_weights: bool = True
    ) -> None:
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((4, 34))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 34, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 50
    for v in cfg:
        if v == 'M':
            continue
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs: Dict[str, List[Union[str, int]]] = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def _vgg(arch: str, cfg: str, batch_norm: bool, pretrained: bool, progress: bool, **kwargs: Any) -> VGG:
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

def vgg16(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 16-layer model (configuration "D")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg16', 'D', False, pretrained, progress, **kwargs)

In [ ]:
#define a network
class Majongai(nn.Module):
  def __init__(self):
    super(Majongai,self).__init__()
    self.model = vgg16(pretrained = False)

  def forward(self,x):
    output = self.model(x)
    return output

In [ ]:
from torchsummary import summary
net = Majongai()
net.to(device)
summary(net,(50,4,34))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 64, 4, 34]          28,864
              ReLU-2            [-1, 64, 4, 34]               0
            Conv2d-3            [-1, 64, 4, 34]          36,928
              ReLU-4            [-1, 64, 4, 34]               0
            Conv2d-5           [-1, 128, 4, 34]          73,856
              ReLU-6           [-1, 128, 4, 34]               0
            Conv2d-7           [-1, 128, 4, 34]         147,584
              ReLU-8           [-1, 128, 4, 34]               0
            Conv2d-9           [-1, 256, 4, 34]         295,168
             ReLU-10           [-1, 256, 4, 34]               0
           Conv2d-11           [-1, 256, 4, 34]         590,080
             ReLU-12           [-1, 256, 4, 34]               0
           Conv2d-13           [-1, 256, 4, 34]         590,080
             ReLU-14           [-1, 256

In [ ]:
import torch.optim as optim
def get_time_dif(start_time):
  end_time = time.time()
  time_dif = end_time - start_time
  return timedelta(seconds=int(round(time_dif)))

In [ ]:
def evaluate(model, data_loader):
  model.eval()
  loss_total = 0
  predict_all = np.array([], dtype=int)
  labels_all = np.array([], dtype=int)
  with torch.no_grad():
    for inputs, labels in data_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = F.cross_entropy(outputs, labels)
      loss_total += loss
      labels = labels.data.cpu().numpy()
      predic = torch.max(outputs.data,1)[1].cpu().numpy()
      labels_all = np.append(labels_all, labels)
      predict_all = np.append(predict_all, predic)
  acc = metrics.accuracy_score(labels_all, predict_all)
  recall = metrics.recall_score(labels_all, predict_all,average='macro')
  f1 = metrics.f1_score(labels_all, predict_all,average='macro')
  return acc, recall, f1, loss_total/len(data_loader)

In [ ]:
def train(model,train_loader,val_loader,learning_rate=0.001,
    num_epochs=100,save_path='/content/drive/MyDrive/统计麻将/result/best_sl.ckpt'):
  start_time = time.time()
  model.train()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  total_batch = 0
  val_best_loss = float('inf')
  last_improve = 0
  flag = False

  trainloss = []
  trainacc = []
  valloss = []
  valacc = []
  valrecall = []
  valf1 = []

  for epoch in range(num_epochs):
    running_loss = 0.0
    print('Epoch[{}/{}]'.format(epoch+1, num_epochs))
    for i,data in enumerate(train_loader,0):
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      if i % 50 == 49:
        true = labels.data.cpu()
        predic = torch.max(outputs.data,1)[1].cpu()
        train_acc = metrics.accuracy_score(true,predic)
        val_acc, val_recall, val_f1, val_loss = evaluate(model,val_loader)

        trainloss.append(running_loss / 50)
        trainacc.append(train_acc)
        valloss.append(val_loss)
        valacc.append(val_acc)
        valrecall.append(val_recall)
        valf1.append(val_f1)

        if val_loss < val_best_loss:
          val_best_loss = val_loss
          torch.save(model.state_dict(),save_path)
          improve = '*'
          last_improve = total_batch
        else:
          improve = ''
        time_dif = get_time_dif(start_time)
        msg = 'Iter: {0:>6}, Train Loss:{1:>5.4}, Train Acc:{2:6.2%}, Val Loss:{3:5.4}, Val Acc:{4:6.2%}, Val Recall:{5:6.2%} Val F1:{6:6.2%}, Time:{7} {8}'
        print(msg.format(total_batch,running_loss / 50,train_acc,val_loss,val_acc,val_recall,val_f1,time_dif,improve))
        model.train()
        running_loss = 0.0
      total_batch += 1
      # if total_batch - last_improve > 10000:
      #   print("No optimization for a long time, auto-stopping...")
      #   flag = True
      #   break
    # if flag:
    #   break                  

In [ ]:
model = Majongai().to(device)
train(model, train_dataloader, val_dataloader,learning_rate=0.001)

Epoch[1/100]
Iter:     49, Train Loss:4.697, Train Acc: 6.25%, Val Loss:3.519, Val Acc: 4.07%, Val Recall: 2.94% Val F1: 0.23%, Time:0:00:31 *
Iter:     99, Train Loss:3.508, Train Acc: 2.34%, Val Loss:3.497, Val Acc: 4.40%, Val Recall: 2.94% Val F1: 0.25%, Time:0:00:59 *
Iter:    149, Train Loss:3.481, Train Acc: 5.47%, Val Loss: 3.48, Val Acc: 4.21%, Val Recall: 2.90% Val F1: 0.37%, Time:0:01:32 *
Iter:    199, Train Loss:3.458, Train Acc: 2.34%, Val Loss:3.441, Val Acc: 4.23%, Val Recall: 3.16% Val F1: 0.42%, Time:0:02:03 *
Iter:    249, Train Loss:3.449, Train Acc: 6.25%, Val Loss: 3.45, Val Acc: 4.38%, Val Recall: 3.56% Val F1: 0.54%, Time:0:02:21 
Iter:    299, Train Loss:3.452, Train Acc: 0.00%, Val Loss: 3.44, Val Acc: 4.32%, Val Recall: 3.54% Val F1: 0.54%, Time:0:02:48 *
Iter:    349, Train Loss:3.448, Train Acc: 4.69%, Val Loss:3.437, Val Acc: 4.74%, Val Recall: 3.53% Val F1: 0.69%, Time:0:03:18 *
Iter:    399, Train Loss:3.437, Train Acc: 5.47%, Val Loss:3.435, Val Acc: 4.6

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
np.save('/content/drive/MyDrive/统计麻将/result/trainloss.npy',np.array(trainloss)) # 保存为.npy格式
np.save('/content/drive/MyDrive/统计麻将/result/trainacc.npy,np'.array(trainacc))
np.save('/content/drive/MyDrive/统计麻将/result/valloss.npy',np.array(valloss))
np.save('/content/drive/MyDrive/统计麻将/result/valacc.npy',np.array(valacc))
np.save('/content/drive/MyDrive/统计麻将/result/valrecall.npy',np.array(valrecall))
np.save('/content/drive/MyDrive/统计麻将/result/valf1.npy',np.array(valf1))

NameError: ignored